In [ ]:
USE {
    dependencies("org.mariadb.jdbc:mariadb-java-client:3.1.4")
}

In [ ]:
%use dataframe, kandy

In [ ]:
import org.jetbrains.kotlinx.dataframe.DataFrame
import org.jetbrains.kotlinx.dataframe.api.describe
import org.jetbrains.kotlinx.dataframe.api.print
import org.jetbrains.kotlinx.dataframe.io.DatabaseConfiguration
import org.jetbrains.kotlinx.dataframe.io.getSchemaForSqlTable
import org.jetbrains.kotlinx.dataframe.io.readSqlTable
import org.jetbrains.kotlinx.dataframe.io.getSchemaForAllSqlTables
import org.jetbrains.kotlinx.dataframe.schema.DataFrameSchema
import java.sql.DriverManager
import java.util.*


In [ ]:
val URL = "jdbc:mariadb://localhost:3307/imdb"
val USER_NAME = "root"
val PASSWORD = "pass"

val dbConfig = DatabaseConfiguration(URL, USER_NAME, PASSWORD)

# Extracting data about Tarantino films from the ResultSet 

In [1]:
import javax.imageio.ImageIO
val image = ImageIO.read(File("../../src/main/resources/images/quentin-tarantino-movies-posters.png"))
DISPLAY(image)

Line_1.jupyter.kts (2:26 - 30) Unresolved reference: File

In [ ]:
import java.util.Properties

val props = Properties()
props.setProperty("user", USER_NAME)
props.setProperty("password", PASSWORD)

val TARANTINO_FILMS_SQL_QUERY = """
    SELECT name, year, rank,
    GROUP_CONCAT (genre) as "genres"
    FROM movies JOIN movies_directors ON movie_id = movies.id
    JOIN directors ON directors.id=director_id LEFT JOIN movies_genres ON movies.id = movies_genres.movie_id
    WHERE directors.first_name = "Quentin" AND directors.last_name = "Tarantino"
    GROUP BY name, year, rank
    ORDER BY year
    """

var dfTarantinoFilmsRs: DataFrame<*>

DriverManager.getConnection(URL, props).use { connection ->
        connection.createStatement().use { st ->
            st.executeQuery(TARANTINO_FILMS_SQL_QUERY).use { rs ->
               val dfTarantinoFilmsSchema = DataFrame.getSchemaForResultSet(rs, connection)
               dfTarantinoFilmsSchema.print()

               dfTarantinoFilmsRs = DataFrame.readResultSet(rs, connection)
               dfTarantinoFilmsRs
            } 
        } 
}

In [ ]:
val df = dfTarantinoFilmsRs
    .fillNA { year }
    .with { 0 }
    .convert { year }.toInt()


In [ ]:
df.filter { year > 2000 }